# KaBOB-MOPs

## AllegroGraph Python Tutorial

Set server variables

## KaBOB Interface Test

KaBOBInterface is used to mopify and parse KaBOB triples. OpenKaBOB opens a connection to KaBOB and closes it using a `with` statement

In [1]:
from KaBOB_Interface import KaBOBInterface, OpenKaBOB
import KaBOB_Constants
with OpenKaBOB() as kabob:
    print('Triples in KaBOB:', kabob.size())

DEBUG:OpenKaBOB:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037
DEBUG:OpenKaBOB:Closed kabob_prod_feb2018


Triples in KaBOB: 283830446


### Basic Querying

#### Without using the interface (using just agraph_python)

In [2]:
from KaBOB_Interface import OpenKaBOB

p53 = kabob.createURI("http://purl.obolibrary.org/obo/PR_P04637")
label = kabob.createURI("http://www.w3.org/2000/01/rdf-schema#label")

with OpenKaBOB() as kabob:
    print('Triples in KaBOB:', kabob.size())
    
    print('Label for p53')
    p53_labels = kabob.getStatements(p53, label).asList()
    p53_label = p53_labels[0].getObject()
    print("\t%s" % p53_label)
    
    print()
    
    print('Triples with p53 as the subject')
    p53_as_subject = kabob.getStatements(p53, None, None)
    with p53_as_subject:
        p53_as_subject.enableDuplicateFilter()  # This may or may not be necessary all the time
        for statement in p53_as_subject:
            object_labels = kabob.getStatements(statement.getObject(), label).asList()
            
            if object_labels:
                print("\t%s %s %s" % (p53_label, statement.getPredicate().getLocalName(), object_labels[0].getObject()))
    
    print()
    
    print('Triples with p53 as the object')
    p53_as_object = kabob.getStatements(None, None, p53)
    with p53_as_object:
        p53_as_object.enableDuplicateFilter()
        for statement in p53_as_object:
            subject_labels = kabob.getStatements(statement.getSubject(), label).asList()
            
            if subject_labels:
                print("\t%s %s %s" % (p53_label, statement.getPredicate().getLocalName(), subject_labels[0].getObject()))


DEBUG:OpenKaBOB:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037


Triples in KaBOB: 283830446
Label for p53
	"cellular tumor antigen p53 (human)"

Triples with p53 as the subject
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53"
	"cellular tumor antigen p53 (human)" subClassOf "Homo sapiens protein"

Triples with p53 as the object
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 isoform 1 (human)"
	"cellular tumor antigen p53 (human)" annotatedSource "PR:P04637 located_in GO:0005654"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 15 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 14 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 dimethylated 3 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 16 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 1 (human)"
	"cel

DEBUG:OpenKaBOB:Closed kabob_prod_feb2018



	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 18 (human)"
	"cellular tumor antigen p53 (human)" subClassOf "cellular tumor antigen p53 phosphorylated 4 (human)"


#### With the interface

In [5]:
from KaBOB_Interface import KaBOBInterface, OpenKaBOB

p53 = "obo:PR_P04637"

with OpenKaBOB() as kabob:
    interface = KaBOBInterface(kabob)
    
    print('Label for p53')
    p53_label = interface.node_print_name(p53)
    print("\t%s" % p53_label)
    
    print()
    
    print('Triples with p53 as the subject')
    for statement in interface.agraph_get_objects(p53, None, full_statement=True):
        object_label = interface.node_print_name(statement.getObject())
        predicate_label = interface.node_print_name(statement.getPredicate())
        print("\t%s %s %s" % (p53_label, predicate_label, object_label))
    
    print()
    
    print('Triples with p53 as the object')
    for statement in interface.agraph_get_subjects(p53, None, full_statement=True):
        subject_label = interface.node_print_name(statement.getSubject())
        predicate_label = interface.node_print_name(statement.getPredicate())
        print("\t%s %s %s" % (subject_label, predicate_label, p53_label))

DEBUG:OpenKaBOB:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037


Label for p53
	cellular tumor antigen p53 (human)

Triples with p53 as the subject
	cellular tumor antigen p53 (human) database_cross_reference "UniProtKB:P04637"
	cellular tumor antigen p53 (human) database_cross_reference "Reactome:R-HSA-69488"
	cellular tumor antigen p53 (human) id "PR:P04637"
	cellular tumor antigen p53 (human) has_obo_namespace "protein"
	cellular tumor antigen p53 (human) definition "A cellular tumor antigen p53 that is encoded in the genome of human."
	cellular tumor antigen p53 (human) label "cellular tumor antigen p53 (human)"
	cellular tumor antigen p53 (human) comment "Category=organism-gene."
	cellular tumor antigen p53 (human) subClassOf cellular tumor antigen p53
	cellular tumor antigen p53 (human) subClassOf Homo sapiens protein
	cellular tumor antigen p53 (human) subClassOf pr_genid1776552
	cellular tumor antigen p53 (human) subClassOf pr_genid338241
	cellular tumor antigen p53 (human) equivalentClass pr_genid338237
	cellular tumor antigen p53 (human) t

DEBUG:OpenKaBOB:Closed kabob_prod_feb2018


	cellular tumor antigen p53 phosphorylated 4 (human) subClassOf cellular tumor antigen p53 (human)


### Mopification

In [6]:
from KaBOB_Interface import KaBOBInterface, OpenKaBOB
import KaBOB_Constants
p53 = "obo:PR_P04637"

with OpenKaBOB() as kabob:
    interface = KaBOBInterface(kabob)

    bio_p53 = interface.bio(interface.create_uri(p53))

    interface.mopify(bio_p53, 0)
    interface.mopsManager.show_frame(interface.node_print_name(bio_p53), max_depth=2)

DEBUG:OpenKaBOB:Connecting to AllegroGraph server --host:'amc-tantor.ucdenver.pvt' port:10037
DEBUG:KaBOBInterface:Mopifying cellular tumor antigen p53 (human)
DEBUG:KaBOBInterface:	Mopifying Restriction - 
DEBUG:KaBOBInterface:	Finished mopifying Restriction - 
DEBUG:KaBOBInterface:	Mopifying Restriction - 
DEBUG:KaBOBInterface:	Finished mopifying Restriction - 
DEBUG:KaBOBInterface:	Mopifying gene product abstraction - 
C:\Users\pielk\Google Drive\HunterLab\KaBOB\Python\KaBOB_Interface.py:48: UserWarning: gene product abstraction -  has both superclasses and type
  warnings.warn(name + " has both superclasses and type")
DEBUG:KaBOBInterface:		Mopifying gene or gene product abstraction - 
DEBUG:KaBOBInterface:		Finished mopifying gene or gene product abstraction - 
DEBUG:KaBOBInterface:		Mopifying gene product or gene product variant abstraction - 
C:\Users\pielk\Google Drive\HunterLab\KaBOB\Python\KaBOB_Interface.py:48: UserWarning: gene product or gene product variant abstraction - 

NAME cellular tumor antigen p53 (human)
TYPE <class 'MOPs.MOP'>
ISA Homo sapiens protein
	NAME Homo sapiens protein
	TYPE <class 'MOPs.MOP'>
	ISA eukaryotic protein
	ROLE label
	FILLER "Homo sapiens protein"@en
	ROLE subClassOf
	FILLER eukaryotic protein
	ROLE equivalentClass
	FILLER BN_pr_genid84260
ISA cellular tumor antigen p53
	NAME cellular tumor antigen p53
	TYPE <class 'MOPs.MOP'>
	ISA protein
	ROLE label
	FILLER "cellular tumor antigen p53"@en
	ROLE subClassOf
	FILLER protein
ISA gene product abstraction - 
	NAME gene product abstraction - 
	TYPE <class 'MOPs.Instance'>
	ISA gene or gene product abstraction - 
	ISA gene product or gene product variant abstraction - 
ROLE label
FILLER "cellular tumor antigen p53 (human)"@en
ROLE subClassOf
FILLER cellular tumor antigen p53
ROLE equivalentClass
FILLER BN_pr_genid338237
ROLE has_gene_template
	NAME Restriction - 
	TYPE <class 'MOPs.Instance'>
ROLE only_in_taxon
	NAME Restriction - 
	TYPE <class 'MOPs.Instance'>


In [ ]:
BP_root = "obo:GO_0008150"
MF_root = "obo:GO_0003674"
CC_root = "obo:GO_0005575"
PRO_root = "obo:PR_000000001"
localization_process = "obo:GO_0051179"
binding_process = "obo:GO_0005488"
interaction = "obo:MI_0000"
physical_association = "obo:MI_0915"
apoptotic_process = "obo:GO_0006915"
p53 = "obo:PR_P04637"
cytochrome_C = "obo:PR_P08574"